In [1]:
import cv2
import mediapipe as mp
import os
import numpy as np

mp_face = mp.solutions.face_detection

def crop_face_mediapipe(image, detection, margin=0.2):
    h, w, _ = image.shape
    
    bbox = detection.location_data.relative_bounding_box
    x, y, ww, hh = bbox.xmin, bbox.ymin, bbox.width, bbox.height

    x1 = int(x * w)
    y1 = int(y * h)
    x2 = int((x + ww) * w)
    y2 = int((y + hh) * h)

    # Apply margin (20% default)
    dx = int(margin * (x2 - x1))
    dy = int(margin * (y2 - y1))

    x1 = max(0, x1 - dx)
    y1 = max(0, y1 - dy)
    x2 = min(w, x2 + dx)
    y2 = min(h, y2 + dy)

    return image[y1:y2, x1:x2]


def extract_frames_with_face(video_path, output_dir, num_frames=5, rotate=False):
    os.makedirs(output_dir, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if total < num_frames:
        frame_indices = list(range(total))
    else:
        step = total // num_frames
        frame_indices = [i * step for i in range(num_frames)]

    saved_paths = []

    with mp_face.FaceDetection(model_selection=0, min_detection_confidence=0.6) as detector:
        for idx in frame_indices:
            cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
            ok, frame = cap.read()
            if not ok:
                continue

            # Fix rotation (README warns about Android videos)
            if rotate:
                frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)

            # Detect face
            results = detector.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

            if not results.detections:
                continue

            # Use first face
            face = crop_face_mediapipe(frame, results.detections[0])

            if face.size == 0:
                continue

            face = cv2.resize(face, (256, 256))

            outname = f"{os.path.basename(video_path)}_f{idx}.jpg"
            outpath = os.path.join(output_dir, outname)
            cv2.imwrite(outpath, face)

            saved_paths.append(outpath)

    cap.release()
    return saved_paths


def extract_MSU_dataset_faces(msu_root, output_root, num_real=5, num_attack=8):
    scene_dirs = [d for d in os.listdir(msu_root) if "scene" in d]

    out_live = os.path.join(output_root, "live")
    out_spoof = os.path.join(output_root, "not_live")
    os.makedirs(out_live, exist_ok=True)
    os.makedirs(out_spoof, exist_ok=True)

    for scene in scene_dirs:
        scene_path = os.path.join(msu_root, scene)
        real_dir = os.path.join(scene_path, "real")
        attack_dir = os.path.join(scene_path, "attack")

        if not os.path.isdir(real_dir):  
            continue  

        print(f"Processing {scene}")

        # REAL videos
        for fname in os.listdir(real_dir):
            if fname.lower().endswith((".mp4", ".mov", ".avi")):
                fpath = os.path.join(real_dir, fname)
                extract_frames_with_face(
                    fpath, out_live, num_frames=num_real,
                    rotate=("android" in fname.lower())
                )

        # ATTACK videos
        for fname in os.listdir(attack_dir):
            if fname.lower().endswith((".mp4", ".mov", ".avi")):
                fpath = os.path.join(attack_dir, fname)
                extract_frames_with_face(
                    fpath, out_spoof, num_frames=num_attack,
                    rotate=("android" in fname.lower())
                )

    print("MSU face extraction completed!")


In [6]:
import cv2
import os
import numpy as np

def load_face_file(face_path):
    """Đọc file .face và trả về dict: frame_id -> (x1, y1, x2, y2)"""
    boxes = {}
    with open(face_path, "r") as f:
        for line in f:
            parts = line.strip().split(",")
            if len(parts) < 5:
                continue
            frame_id = int(parts[0])
            x1, y1, x2, y2 = map(int, parts[1:5])
            boxes[frame_id] = (x1, y1, x2, y2)
    return boxes


def extract_MSU_face_frames(video_path, face_path, output_dir, max_frames=6):
    os.makedirs(output_dir, exist_ok=True)

    face_boxes = load_face_file(face_path)
    if len(face_boxes) == 0:
        # Không có annotation nào cho video này
        return []

    # 👉 LẤY DANH SÁCH FRAME CÓ BOX TỪ FILE .face
    all_fids = sorted(face_boxes.keys())

    # Chọn tối đa max_frames frame rải đều trong số frame có box
    if len(all_fids) <= max_frames:
        selected_fids = all_fids
    else:
        idxs = np.linspace(0, len(all_fids) - 1, max_frames, dtype=int)
        selected_fids = [all_fids[i] for i in idxs]

    cap = cv2.VideoCapture(video_path)
    saved = []

    for fid in selected_fids:
        # Đặt con trỏ video tới frame fid
        cap.set(cv2.CAP_PROP_POS_FRAMES, fid)
        ok, frame = cap.read()
        if not ok:
            continue

        x1, y1, x2, y2 = face_boxes[fid]

        # Crop face từ bounding box
        face = frame[y1:y2, x1:x2]
        if face.size == 0:
            continue

        # Resize về 256x256
        face = cv2.resize(face, (256, 256))

        # Lưu file
        outname = f"{os.path.basename(video_path)}_f{fid}.jpg"
        outpath = os.path.join(output_dir, outname)
        cv2.imwrite(outpath, face)
        saved.append(outpath)

    cap.release()
    return saved

def extract_MSU_dataset_by_face(msu_root, output_root, max_frames_real=5, max_frames_attack=8):
    os.makedirs(output_root, exist_ok=True)
    out_live = os.path.join(output_root, "live")
    out_spoof = os.path.join(output_root, "not_live")
    os.makedirs(out_live, exist_ok=True)
    os.makedirs(out_spoof, exist_ok=True)

    for root, dirs, files in os.walk(msu_root):
        for fname in files:
            if fname.endswith(".mp4") or fname.endswith(".avi") or fname.endswith(".mov"):
                video_path = os.path.join(root, fname)

                # File .face ứng với video
                face_path = video_path.replace(".mp4", ".face").replace(".avi", ".face").replace(".mov", ".face")
                if not os.path.exists(face_path):
                    print("Missing face file:", face_path)
                    continue

                # live hay spoof?
                if "real" in root.lower():
                    out_dir = out_live
                    max_frames = max_frames_real
                else:
                    out_dir = out_spoof
                    max_frames = max_frames_attack

                extract_MSU_face_frames(
                    video_path,
                    face_path,
                    out_dir,
                    max_frames=max_frames
                )

    print(">>> MSU face extraction done (using .face GT bounding boxes)!")


In [7]:
extract_MSU_dataset_by_face(
    msu_root="MSU-MFSD",          
    output_root="MSU_MFSD_faces",  
    max_frames_real=5,
    max_frames_attack=8
)

>>> MSU face extraction done (using .face GT bounding boxes)!
